In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from TinyImageNet import TinyImageNet

from advertorch.attacks import LinfPGDAttack, GradientSignAttack, LinfBasicIterativeAttack, CarliniWagnerL2Attack, MomentumIterativeAttack, SpatialTransformAttack, LinfSPSAAttack, JacobianSaliencyMapAttack

#32 by 32 image size
#from models.vgg_adv import VGG

#64 by 64 image size
from models.vgg_tiny_adv import VGG

In [ ]:
augmentation = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(64)], p=.8)

training_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    transforms.ToTensor()])

valid_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor()])


In [ ]:
root ="tiny-imagenet-200"
in_memory = False
training_set = TinyImageNet(root, 'train', transform=training_transform, in_memory=in_memory)
valid_set = TinyImageNet(root, 'val', transform=valid_transform, in_memory=in_memory)
trainloader = DataLoader(training_set, batch_size=256, shuffle=True, num_workers=8,pin_memory=True)
validloader = DataLoader(valid_set, batch_size=32, num_workers=8,pin_memory=True)

In [ ]:
device = torch.device("cuda")
model = VGG('VGG13').to(device)

In [ ]:
model.load_state_dict(torch.load('vgg13_model_CE.pth'))

In [ ]:
pgd01 = LinfPGDAttack(
    model, eps=0.005,
    nb_iter=10, eps_iter=0.005/10, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)
pgd02 = LinfPGDAttack(
    model, eps=0.01,
    nb_iter=10, eps_iter=0.01/10, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)
pgd03 = LinfPGDAttack(
    model, eps=0.015,
    nb_iter=10, eps_iter=0.015/10, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)


MIM1 = MomentumIterativeAttack(model,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.005, nb_iter=10, eps_iter=0.005/10, clip_min=0.0, clip_max=1.0, targeted=False)
MIM2 = MomentumIterativeAttack(model,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.01, nb_iter=10, eps_iter=0.01/10, clip_min=0.0, clip_max=1.0, targeted=False)
MIM3 = MomentumIterativeAttack(model,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.015, nb_iter=10, eps_iter=0.015/10,clip_min=0.0, clip_max=1.0, targeted=False)


SPSA = LinfSPSAAttack(model,0.015,nb_iter=10)

In [ ]:
attacks = {
          'pgd0.1':pgd01, 
          'pgd0.2':pgd02,
          'pgd0.3':pgd03,

          'MIM0.1': MIM1,
          'MIM0.2': MIM2,
          'MIM0.3': MIM3,
           'SPSA': SPSA
           }

In [ ]:
model.eval()
for attack_name in attacks:
    correct = 0
    size = 0 
    n = 0
    for tedata, tetarget in validloader:
        n += 1
        size += tedata.shape[0]
        tedata, tetarget = tedata.to(device), tetarget.to(device)

        tedata = attacks[attack_name].perturb(tedata, tetarget).to(device)
        output = model(tedata)
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(tetarget.view_as(pred)).sum().item()

    print("{:s} acc: {:.2f}".format(attack_name, 100. * correct / size)) 

In [ ]:
model.load_state_dict(torch.load('vgg13_model_PC_loss_995.pth'))

In [ ]:
attacks = {
          'pgd0.1':pgd01, 
          'pgd0.2':pgd02,
          'pgd0.3':pgd03,

          'MIM0.1': MIM1,
          'MIM0.2': MIM2,
          'MIM0.3': MIM3,
           'SPSA': SPSA
           }

In [ ]:
model.eval()
for attack_name in attacks:
    correct = 0
    size = 0 
    n = 0
    for tedata, tetarget in validloader:
        n += 1
        size += tedata.shape[0]
        tedata, tetarget = tedata.to(device), tetarget.to(device)

        tedata = attacks[attack_name].perturb(tedata, tetarget).to(device)
        output = model(tedata)
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(tetarget.view_as(pred)).sum().item()

    print("{:s} acc: {:.2f}".format(attack_name, 100. * correct / size)) 